In [1]:
from binance.client import Client
import sqlalchemy
import pandas as pd
import numpy as np
import math
import os
from binance.exceptions import BinanceAPIException
import time
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import talib

In [2]:
api_key='MymKgk3yTXUkIac2KzrQqGKXfvtFcYNiPtaWmnAd3LWyhKJhS4JXiYR5we8w6EJG' #leo
api_secret='mBDa4JTLIH34Ys8n1qg4LLhJ6ZzN1CigGlVYWDnuVZhAqBoGfDsSsKEnKWZIZgBb'

api_key2='Gu9ivmske8eTf0Z5an7DgCTD0XYPdjjzDQ4tpGw7laiaGlkDR1suXU2O7yKOmkDe' #arthur
api_secret2='BeNGcapE19jdFyIdKZufpnLlcfOlFYzGS4JJf8egXeVtRCzdJvXuib0vQ7t1IryX'

client=Client(api_key, api_secret)
client2=Client(api_key2, api_secret2)

In [3]:
pair = "ETHUSDT"

In [4]:
engine = sqlalchemy.create_engine('sqlite:///'+pair+'stream.db')

In [5]:
def sendmail(msg):
    sender_address = 'leo.lyu10@gmail.com'
    sender_pass = 'Lty7959123'
    receiver_address = 'llyu0@eq.edu.au'
    mail_content = str(msg)
    
    message = MIMEMultipart() 
    message['From'] = sender_address 
    message['To'] = receiver_address 
    message['Subject'] = 'Trading Bot'

    message.attach(MIMEText(mail_content, 'plain'))

    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()

In [6]:
def get5minutedata(pair):
    
    #ETHUSDT, 5m, 1100m
    
    #防止连接币安错误，60ms
    try:
        df=pd.DataFrame(client2.futures_historical_klines(pair,'5m','1100m UTC'))
    except: #BinanceAPIException as e:
        #print(e)
        #time.sleep(60)
        df=pd.DataFrame(client.futures_historical_klines(pair,'5m','1100m UTC'))

    df=df.iloc[:,:12]
    df.columns=['Time','Open','High','Low','Close','Volume','VWAP1','VWAP2','BBand_Upper','BBand_Middle','BBand_Lower','ATR']
    df=df.set_index("Time")
    df.index=pd.to_datetime(df.index, unit='ms')
    df=df.astype(float)
    
    #VWAP 200根 现在
    df['VWAP1']=(df.iloc[19:,4]*(df.iloc[19:,0]+df.iloc[19:,1] + df.iloc[19:,2])/3).cumsum()/df.iloc[19:,4].cumsum()

    #VWAP 200根 100min ago
    df['VWAP2']=(df.iloc[:200,4]*(df.iloc[:200,0]+df.iloc[:200,1] + df.iloc[:200,2])/3).cumsum()/df.iloc[:200,4].cumsum()   

    #布林带，周期50，标准差倍数3
    df['BBand_Upper']=df['Close'].rolling(window=50).mean()+(df['Close'].rolling(window=50).std()*3)
    df['BBand_Middle']=df['Close'].rolling(window=50).mean()
    df['BBand_Lower']=df['Close'].rolling(window=50).mean()-(df['Close'].rolling(window=50).std()*3)
    df['ATR']= talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)

    return df

In [7]:
def websocket1minute(): 
    while True:
        tp = pd.read_sql(pair, engine)
        return tp.iloc[-1,1]

In [8]:
def tradingstrategy(Symbol, Buy = False, Sell = False):
    print(datetime.datetime.now())
    print('Start（不要忘了运行‘1minute close engine! ）')
    
    client.futures_change_leverage(symbol=Symbol, leverage=1) #调整杠杆
    g = client.futures_account_balance() #检查合约账户余额 
    h = g[2]['balance']
    print('保证金余额(合约): ' + h +' BUSD' )
    print(' ')
    
    while True:
        
        #price_meet_times = 0
        nowtime = time.localtime()
        minute = nowtime[4]
        second = nowtime[5]
       
            
        if not Buy and not Sell\
        and second == 55 and minute % 10 == 4\
        or second == 55 and minute % 10 == 9:
            
            
            df = get5minutedata(pair)
            open_position_price = df.iat[219,3]
            open_position_ATR = df.iat[219,10]
            
            
            vwap =  df.iat[219,5]
            vwap_slope = (df.iat[219,5] - df.iat[199,6]) / 20
            ma_20 = df.iloc[199:,3].mean()
            price_1k_ago=df.iat[218,3]
            ma_3 = df.iloc[217:,3].mean()
            ma_3_1k_ago = df.iloc[216:219,3].mean()
            ma_3_2k_ago = df.iloc[215:218,3].mean()
            
            #开多
            if  vwap_slope > -0.01\
            and ma_3 >= vwap\
            and ma_3_1k_ago < vwap\
            and ma_3_2k_ago <vwap\
            and vwap > ma_20\
            and ((df.iat[219,7] - df.iat[219,9]) / df.iat[219,8]) <= 0.08\
            and ((df.iat[218,7] - df.iat[218,9]) / df.iat[218,8]) <= 0.08\
            and ((df.iat[217,7] - df.iat[217,9]) / df.iat[217,8]) <= 0.08:
                
                
                a = client.futures_account_balance() #检查合约账户余额 
                b = a[2]['balance']  #BUSD余额
                b = float(b)
                qty = b / open_position_price #数量=余额/开仓（当前）价格
                qty = int(qty *1000)/1000 #数量保留3位小数 no rounding
                
                
                initial_stoploss_price = open_position_price * 0.99 - open_position_ATR
                print(initial_stoploss_price)
                
                client.futures_cancel_all_open_orders(symbol=Symbol)
                
                client.futures_create_order(symbol=Symbol, 
                              type='MARKET', 
                              side='BUY', 
                              quantity=qty)
                
                print(datetime.datetime.now())
                print('Position_Opened_Buy/Long: ' + str(open_position_price))
                
                client.futures_create_order(symbol=Symbol, 
                                            type='STOP_MARKET',
                                            timeInForce='GTC', 
                                            side='SELL', 
                                            stopPrice=round(initial_stoploss_price, 2), closePosition=True)
                
                print('Initial Stop Prcie: ' + str(round(initial_stoploss_price, 2)))
                Buy=True
                              
            #开空    
            elif vwap_slope < 0.01\
            and ma_3 <= vwap\
            and ma_3_1k_ago > vwap\
            and ma_3_2k_ago > vwap\
            and vwap < ma_20\
            and ((df.iat[219,7] - df.iat[219,9]) / df.iat[219,8]) <= 0.08\
            and ((df.iat[218,7] - df.iat[218,9]) / df.iat[218,8]) <= 0.08\
            and ((df.iat[217,7] - df.iat[217,9]) / df.iat[217,8]) <= 0.08:
                
                client.futures_cancel_all_open_orders(symbol=Symbol)

                a = client.futures_account_balance() #检查合约账户余额 
                b = a[2]['balance']  #BUSD余额
                b = float(b)
                
                qty = b / open_position_price #数量=余额/开仓（当前）价格
                qty = int(qty *1000)/1000 #数量保留3位小数
                
                initial_stoploss_price = open_position_price * 1.01+ open_position_ATR
                
                client.futures_create_order(symbol=Symbol, 
                              type='MARKET', 
                              side='SELL', 
                              quantity=qty)
                      
                print(datetime.datetime.now())
                print('Position_Opened_Sell/Short' + str(open_position_price))
                

                
                client.futures_create_order(symbol=Symbol, 
                            type='STOP_MARKET', 
                            timeInForce='GTC', 
                            side='BUY', stopPrice=round(initial_stoploss_price, 2), closePosition=True)
                
                print('Initial Stop Prcie: ' + str(round(initial_stoploss_price, 2)))
                Sell=True
                
                
            if not Buy and not Sell:
                print(datetime.datetime.now())
                print('Condition Not Meet')
                print(' ')
                print(' ')
                
                time.sleep(30)
        
        while Buy:
            
            buy_nowtime = time.localtime()
            buy_minute = buy_nowtime[4]
            buy_second = buy_nowtime[5]

            if  buy_second >= 55 and buy_minute % 10 == 4\
            or buy_second >= 55 and buy_minute % 10 == 9:
                
                df5 = get5minutedata(pair)
                ATR = df5.iloc[-1,10]
                now_5min_close = df5.iloc[-1,3]
                original_stoploss = now_5min_close * 0.99 - ATR
                #now_stoploss_price = now_5min_close - ((now_5min_close - (now_5min_close * 0.99 - ATR)) * (1-(0.1*price_meet_times)))
                now_stoploss_price = now_5min_close * 0.99 - ATR
                
                price_target = now_5min_close + (now_5min_close - original_stoploss) *1.5
                
                client.futures_create_order(symbol=Symbol, 
                            type='STOP_MARKET', 
                            timeInForce='GTC', 
                           side='SELL', stopPrice=round(now_stoploss_price, 2), closePosition=True)
                
                print('Stop Price: '+str(now_stoploss_price))
                print('Price Target: '+str(price_target))
                
                msg ='Price Target: '+str(price_target)
                sendmail('Position_Opened_Buy/Long: ' + str(open_position_price))
                sendmail('Initial Stop Prcie: ' + str(round(initial_stoploss_price, 2)))
                sendmail(msg)
                
                
                while True:

                    buy_stop_nowtime = time.localtime()
                    buy_stop_second = buy_stop_nowtime[5]

                    if buy_stop_second >= 56:
                        minute1_close = websocket1minute()
                        
                        if minute1_close >= price_target:
                            #price_meet_times = price_meet_times+1

                            print(datetime.datetime.now())
                            print('Price Target Meet')

                            break

                        elif minute1_close <= now_stoploss_price\
                        or minute1_close <= initial_stoploss_price:

                            Buy = False

                            print(datetime.datetime.now())
                            print('Position Closed Stoploss Buy/Long: ' +str(minute1_close))
                            
                            profit = ((minute1_close - open_position_price) / open_position_price * 100) - 0.01
                            profit = str(round(profit, 2))+'%'
                            sendmail(profit)
                            
                            print('Profit: ' + profit)
                            print(' ')
                            time.sleep(900)
                            
                            break
                        
                        time.sleep(30)    
                        
    
        while Sell:
            
            sell_nowtime = time.localtime()
            sell_minute = sell_nowtime[4]
            sell_second = sell_nowtime[5]
            
            if  sell_second >= 55 and sell_minute % 10 == 4\
            or sell_second >= 55 and sell_minute % 10 == 9:
                
                df5=get5minutedata(pair)
                
                ATR = df5.iat[219,10]
                now_5min_close = df5.iloc[-1,3]
                original_stoploss = now_5min_close * 1.01 + ATR
                #now_stoploss_price = (((now_5min_close * 1.01 + ATR) - now_5min_close) * (1-(0.1*price_meet_times))) + now_5min_close 
                now_stoploss_price = now_5min_close * 1.01 + ATR
                
                price_target = now_5min_close - (original_stoploss - now_5min_close) *1.5
            
                

                client.futures_create_order(symbol=Symbol, 
                                            type='STOP_MARKET', 
                                            timeInForce='GTC', 
                                            side='BUY', stopPrice=round(now_stoploss_price, 2), closePosition=True)
                
                print('Stop Price: '+str(now_stoploss_price))
                print('Price Target: '+str(price_target))
                
                msg ='Price Target: '+str(price_target)
                sendmail('Position_Opened_Sell/Short: ' + str(open_position_price))
                sendmail('Initial Stop Prcie: ' + str(round(initial_stoploss_price, 2)))
                sendmail(msg)

                while True:

                    sell_stop_nowtime = time.localtime()
                    sell_stop_second = sell_stop_nowtime[5]


                    if sell_stop_second >= 56:

                        minute1_close = websocket1minute()

                        if minute1_close <= price_target:
                            price_meet_times = price_meet_times+1
                            print(datetime.datetime.now())
                            print('Price Target Meet')
                            break

                        elif minute1_close >= now_stoploss_price\
                        or minute1_close >= initial_stoploss_price:

                            Sell = False
                            print(datetime.datetime.now())
                            print('Position Closed Stoploss Sell/Short: ' +str(minute1_close))

                            
                            profit = ((open_position_price - minute1_close) / open_position_price * 100) - 0.1
                            profit = str(round(profit, 2))+'%'
                            sendmail(profit)
                            
                            print('Profit: ' + profit)
                            print(' ')
                            time.sleep(900)

                            break

                        time.sleep(30)                    

In [ ]:
tradingstrategy('ETHBUSD')

2021-09-26 01:19:07.881203
Start（不要忘了运行‘1minute close engine! ）
保证金余额(合约): 37.79108216 BUSD
 
2021-09-26 01:19:55.861091
Condition Not Meet
 
 
2021-09-26 01:24:55.510500
Condition Not Meet
 
 
2021-09-26 01:29:55.484488
Condition Not Meet
 
 
2021-09-26 01:34:55.505871
Condition Not Meet
 
 
2021-09-26 01:39:55.521218
Condition Not Meet
 
 
2021-09-26 01:44:55.512310
Condition Not Meet
 
 
2021-09-26 01:49:55.894212
Condition Not Meet
 
 
2021-09-26 01:54:55.474126
Condition Not Meet
 
 
2021-09-26 01:59:55.477793
Condition Not Meet
 
 
2021-09-26 02:04:55.467399
Condition Not Meet
 
 
2021-09-26 02:09:55.506564
Condition Not Meet
 
 
2021-09-26 02:14:55.459443
Condition Not Meet
 
 
